**Find Mininmum Distance from a rental property to any point of interest**

In [51]:
# Import necessary libraries
from pyspark.sql import SparkSession, functions as F
import geopandas as gpd
from shapely.geometry import Point

# Create Spark session
spark = (
    SparkSession.builder.appName('Weekly Rental Listings Preprocessing')
    .config("spark.sql.repl.eagerEval.enabled", True)  # display full dataframe in console
    .config("spark.sql.parquet.cacheMetadata", "true") # cache parquet metadata
    .config("spark.sql.session.timeZone", "Etc/UTC") # set timezone to UTC
    # memory configurations - hopefully will reduce crashing
    .config("spark.driver.memory", "4g") # set driver memory
    .config("spark.executor.memory", "4g") # set executor memory
    .getOrCreate()
)

In [52]:
sdf

age_0_to_19,age_20_to_39,age_40_to_59,age_60_plus,agency_name,agent_name,appointment_only,avg_days_on_market,bathrooms,bedrooms,car_spaces,description,family_percentage,features_list,first_listed_date,full_address,image_urls,inspection_text,land_area,last_sold_date,latitude,listing_status,listing_tag,listing_url,long_term_resident,longitude,median_rent_price,median_sold_price,number_of_photos,number_sold,owner_percentage,postcode,property_features,property_id,property_type,rental_price,renter_percentage,schools,single_percentage,state_abbreviation,street,street_number,structured_features,suburb,unit_number,updated_date,url
0.142523363,0.5,0.226635516,0.130841121,RT Edgar - Northside,Lily Passarelli,False,60,NULL,3,2,Just freshly pain...,0.5304348,Split System Air ...,2025-08-26T16:14:...,14 Federation Lan...,https://rimh2.dom...,NULL,NULL,NULL,-37.796893,live,NULL,https://www.domai...,0.4220963,144.9961565,900,1350000,11,34,0.5895954,3067,"3, ,2, ,2,",17732837,Townhouse,$880 per week,0.410404623,"""[('Clifton Hill ...",'Catholic','primary',945.4572444065353),('Fitzroy High S...,'Government','secondary',1213.908009865201),('Fitzroy High S...,'Government'
0.08416759,0.6407119,0.200222462,0.0748980343,Fletchers Project...,Joyee Jiang,False,69,NULL,2,2,Perfectly positio...,0.496447623,NULL,2025-09-02T17:45:...,106/609 Victoria ...,https://rimh2.dom...,NULL,NULL,NULL,-37.8110653,live,NULL,https://www.domai...,0.44155845,145.0076834,660,580000,14,187,0.2623574,3067,"2, ,2, ,1,",17744154,Apartment / Unit ...,$750 per week,0.7376426,"""[('Yarra Primary...",'Catholic','combined',1446.984075621626),('Ignatius Learn...,'Catholic','secondary',1490.3809851499548),('Melbourne Indi...,'Private'
0.123569794,0.3340961,0.23569794,0.306636155,RentBetter,NULL,True,85,NULL,1,1,PROPERTY ID: 3999...,0.40625,"Air conditioning,...",2025-09-06T22:53:...,Abbotsford VIC 3067,https://rimh2.dom...,NULL,NULL,NULL,-37.80210950000001,live,NULL,https://www.domai...,0.491228074,145.0019064,510,393000,15,141,0.4144737,3067,"1, ,1, ,1,",17750349,Apartment / Unit ...,$630 per week,0.5855263,"""[('Sophia Mundi ...",'Catholic','primary',1375.7403488678867),('Lynall Hall Co...,'Government','secondary',1597.8921921996487),('Trinity Cathol...,'Catholic'
0.174757287,0.5080906,0.220064729,0.09708738,Miles Real Estate...,Carly Tomat,False,32,NULL,2,1,FIRST INSPECTIONS...,0.44921875,"Air conditioning,...",2025-09-01T10:37:...,4/2 Princes Stree...,https://rimh2.dom...,NULL,NULL,NULL,-37.8092053,recentlyUpdated,NULL,https://www.domai...,0.558935344,144.999856,725,1100000,7,42,0.495575249,3067,"2, ,1, ,1,",17739910,Townhouse,$650 per week,0.504424751,"""[('Abbotsford Pr...",'Catholic','combined',1364.561980898125),('Melbourne Indi...,'Private','secondary',1388.4183229789794),('Ignatius Learn...,'Catholic'
0.170111284,0.400635928,0.275039762,0.154213041,Jellis Craig Rich...,Nathan Roberts,False,32,NULL,2,1,"""Discover this sp...",visit the proper...,0.43849206,"Air conditioning,...",2025-09-08T12:31:...,"45 Park St, Abbot...",https://rimh2.dom...,NULL,NULL,NULL,-37.8080424,new,NULL,https://www.domai...,0.4714829,144.9939399,725,1100000,11,42,0.607142866,3067,"2, ,1, ,−,",17751219,House,$750 per week,0.392857134,"""[('Abbotsford Pr...",'Catholic','primary',1072.966235209828),('Lynall Hall Co...,'Government','secondary',1188.8863983949816),('Richmond High ...
0.06559406,0.6311881,0.225247532,0.0779702961,Dingle Partners,Shane Dangen,True,69,NULL,2,1,"""Be first to insp...",this luxury Abbo...,including gas co...,electric oven an...,large light and ...,spacious bedroom...,luxurious bathro...,and European sty...,stunning polishe...,carpeted bedrooms,split system hea...,intercom entrance,lift access,undercover basem...,0.5532234,Furnished,2025-07-28T14:38:...,302/14 Trenerry C...,https://rimh2.dom...,NULL,NULL,NULL,-37.7996262,live,NULL,https://www.domai...,0.200892851,145.0020093,660,580000,14,187,0.274939179,3067,"2, ,1, ,1,",17688466
0.08416759,0.6407119,0.200222462,0.0748980343

In [89]:
# Load CSV data into Spark DataFrame
sdf = spark.read.option("escape", '"').csv('../data/raw/domain/rental_listings_2025_09.csv', header=True, inferSchema=True)

# Show the first rows full_address column data 

sdf.select("full_address").limit(10)

full_address
14 Federation Lan...
106/609 Victoria ...
Abbotsford VIC 3067
4/2 Princes Stree...
"45 Park St, Abbot..."
302/14 Trenerry C...
DG04/18 Grosvenor...
Y102/125 Turner S...
204/11 Flockhart ...
207 Langridge Str...


In [54]:
# Load the shapefile (use the .shp file as entry point)
sf = gpd.read_file("../data/geo/shpfile/LOCALITY_POLYGON.shp")
sf = sf.to_crs(epsg=4326)  # Convert to lat/lon (WGS84)
sf = sf[["LOCALITY", "geometry"]]
sf["LOCALITY"] = sf["LOCALITY"].str.strip()  # Remove whitespace    
sf["suburb"] = sf["LOCALITY"].str.lower().str.strip()
sf.head()

,LOCALITY,geometry,suburb
0,MOLLONGGHIP,"POLYGON ((144.06544 -37.48382, 144.06438 -37.4...",mollongghip
1,NORTH BLACKWOOD,"POLYGON ((144.38037 -37.42376, 144.38126 -37.4...",north blackwood
2,BASALT,"POLYGON ((144.1184 -37.31148, 144.1184 -37.311...",basalt
3,LLANELLY,"POLYGON ((143.81737 -36.75048, 143.81679 -36.7...",llanelly
4,MURRABIT WEST,"POLYGON ((143.87075 -35.49319, 143.86939 -35.4...",murrabit west


In [55]:
# Function to get suburb name from lon/lat
def get_suburb(lon, lat, sf):
    pt = Point(lon, lat) # e.g., Point(144.9631, -37.8136)
    for _, row in sf.iterrows():
        poly = row['geometry']
        if poly.covers(pt):
            return row['suburb'] # return lower case suburb name
    return None

# try out the function
sub = get_suburb(144.9961565, -37.796893, sf)
print(sub)

abbotsford


In [68]:
school_locations_sf = spark.read.csv('../data/landing/schools/school_locations_2025.csv', header=True, inferSchema=True)
school_locations_sf.show(5)

+----------------+-----------+---------+--------------------+-----------+-------------+--------------------+--------------+--------------+-------------+----------------+---------------------+---------------------+--------------+------------+---------------+-------------+--------------------+--------------------+------+----------------+---------+----------+----------+
|Education_Sector|Entity_Type|School_No|         School_Name|School_Type|School_Status|      Address_Line_1|Address_Line_2|  Address_Town|Address_State|Address_Postcode|Postal_Address_Line_1|Postal_Address_Line_2|   Postal_Town|Postal_State|Postal_Postcode|Full_Phone_No|              Region|                Area|LGA_ID|        LGA_Name| LGA_TYPE|         X|         Y|
+----------------+-----------+---------+--------------------+-----------+-------------+--------------------+--------------+--------------+-------------+----------------+---------------------+---------------------+--------------+------------+---------------+---

In [82]:
def iter_rows(df, columns=None, stream=True):
    """
    Unified row iterator for Pandas/GeoPandas and PySpark DataFrames.
    Yields dicts. Optionally select a subset of `columns`.
    - stream=True uses toLocalIterator() for Spark (memory-safe).
    """
    mod = type(df).__module__

    # --- PySpark DataFrame ---
    if mod.startswith("pyspark.sql"):
        it = df.toLocalIterator() if stream else df.collect()
        for row in it:
            d = row.asDict(recursive=True)
            yield {k: d[k] for k in (columns or d.keys())}

    # --- Pandas / GeoPandas DataFrame ---
    else:
        cols = list(columns) if columns else list(df.columns)
        # itertuples is faster than iterrows; yields namedtuples
        for tup in df.itertuples(index=False, name=None):
            # name=None => plain tuples
            yield dict(zip(df.columns, tup)) if columns is None else \
                {c: v for c, v in zip(df.columns, tup) if c in cols}

In [83]:
# Function that finds the closest POI to a given rental listing
def find_closest_poi_sf(rental_lon, rental_lat, poi_df):
    rental_sub = get_suburb(rental_lon, rental_lat, sf)
    rental_point = Point(rental_lon, rental_lat)
    min_distance = float('inf') # Initialize with infinity
    closest_poi = None

    if rental_sub is None:
        return None, None  # Rental listing not in any suburb
    for poi in iter_rows(poi_df, columns=['School_Name', 'X', 'Y']):
        poi_sub = get_suburb(poi['X'], poi['Y'], sf)
        if poi_sub != rental_sub:
            continue  # Skip POIs not in the same suburb
        poi_point = Point(poi['X'], poi['Y'])
        distance = rental_point.distance(poi_point)
    
        if distance < min_distance:
            min_distance = distance
            closest_poi = poi
            
    return closest_poi, min_distance

# Example usage
rental_lon = 144.9961565
rental_lat = -37.796893
closest_poi, distance = find_closest_poi_sf(rental_lon, rental_lat, school_locations_sf)
if closest_poi:
    print(f"Closest POI: {closest_poi['School_Name']} at distance {distance}")

Closest POI: Sophia Mundi Steiner School at distance 0.011317426087662855


In [84]:
# Read in geojson file with geopandas
ptv_stops_sf = gpd.read_file("../data/landing/ptv/public_transport_stops.geojson")
ptv_stops_sf = ptv_stops_sf.to_crs(epsg=4326)  # Convert to lat/lon (WGS84)
# return the X and Y coordinates of the geometry column
ptv_stops_sf["X"] = ptv_stops_sf["geometry"].x
ptv_stops_sf["Y"] = ptv_stops_sf["geometry"].y
ptv_stops_sf

,STOP_ID,STOP_NAME,MODE,geometry,X,Y
0,17204,Wallan Station,REGIONAL TRAIN,POINT (145.00537 -37.41686),145.005372,-37.416861
1,19980,Melton Station,REGIONAL TRAIN,POINT (144.57222 -37.70336),144.572216,-37.703359
2,19981,Rockbank Station,REGIONAL TRAIN,POINT (144.65071 -37.72919),144.650713,-37.729191
3,19982,Deer Park Station,REGIONAL TRAIN,POINT (144.77083 -37.77727),144.770831,-37.777267
4,19998,Sunbury Station,REGIONAL TRAIN,POINT (144.72803 -37.57915),144.728030,-37.579150
...,...,...,...,...,...,...
29197,6586,Northcott St/Exford Rd,REGIONAL BUS,POINT (144.5743 -37.70666),144.574297,-37.706664
29198,6642,Northcott St/Exford Rd,REGIONAL BUS,POINT (144.57443 -37.70668),144.574433,-37.706676
29199,6644,Exford Rd/Staughton St,REGIONAL BUS,POINT (144.56982 -37.7024),144.569820,-37.702400
29200,7009,Harrison St/Marengo Crst,REGIONAL BUS,POINT (143.66323 -38.77712),143.663226,-38.777123


In [ ]:
def find_closest_poi_gdf(rental_lon, rental_lat, poi_df):
    rental_sub = get_suburb(rental_lon, rental_lat, sf)
    rental_point = Point(rental_lon, rental_lat)
    min_distance = float('inf') # Initialize with infinity
    closest_poi = None

    if rental_sub is None:
        return None, None  # Rental listing not in any suburb
    for poi in iter_rows(poi_df, columns=['STOP_NAME', 'X', 'Y']):
        poi_sub = get_suburb(poi['X'], poi['Y'], sf)
        if poi_sub != rental_sub:
            continue  # Skip POIs not in the same suburb
        poi_point = Point(poi['X'], poi['Y'])
        distance = rental_point.distance(poi_point)
    
        if distance < min_distance: # returns the first closest POI
            min_distance = distance
            closest_poi = poi
            
    return closest_poi, min_distance

# Example usage
rental_lon = 144.9961565
rental_lat = -37.796893
closest_poi, distance = find_closest_poi_gdf(rental_lon, rental_lat, ptv_stops_sf)
if closest_poi:
    print(f"Closest POI: {closest_poi['STOP_NAME']} at distance {distance}")

Closest POI: Park & Ride at distance 0.001689124402682869


In [87]:
closest_poi

{'STOP_NAME': 'Park & Ride', 'X': 144.99492436, 'Y': -37.79804841}